In [ ]:
# for ObjectDetect, import the 'ObjectDetector' function from jetbot module
from jetbot import ObjectDetector

#'ssd_mobilenet_v2_coco.engine' dataset is needed to build the pytorch model
#before running this code, you have to download the 'ssd_mobilenet_v2_coco.engine' 
#this model can detect 90 different common objects
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
# import the 'Camera' function from jetbot module
from jetbot import Camera

# make the camera instance
camera = Camera.instance(width=300, height=300)

In [ ]:
#initialize the image number and object number
image_number = 0
object_number = 0

In [ ]:
import numpy as np
for i in np.arange(0,5):
    #the model receive the image value of the camera instance and save it to 'detections'
    detections = model(camera.value)
    
    #this code prints the results of the detection as the label.
    #if you want to know which label means which object, refer to this link: https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt
    print(detections[image_number][object_number]['label'])

In [ ]:
import sys
#to use the snowboy, please add the path that contains snowboydecoder.py file.
sys.path.append('/home/jetbot/snowboy/examples/Python3')
import snowboydecoder
import signal
from jetbot import Robot
import time
#import the pydub to play the .wav files
from pydub import AudioSegment
from pydub.playback import play
import FaBo9Axis_MPU9250

In [ ]:
#make the robot instance
robot = Robot()
#in  detector.start() function, detected_callback receives 'interrupted'
#if interrupted=True, then the loop of detector.start() is terminated
interrupted = False 

In [ ]:
def signal_handler(signal, frame):
    global interrupted
    interrupted = True

def interrupt_callback():
    global interrupted
    return interrupted

In [ ]:
#pmdl models for snowboy hotward detection
#howmuch_zzan was trained by the sentence- 'how much cheers?(몇 잔이야?)'
#zzanhalgga was trained by the sentence- 'shall we cheers?(우리 짠할까?)'
#these models were trained in Korean
models = ['howmuch_zzan.pmdl', 'zzanhalgga.pmdl']

In [ ]:
#play the .wav file according to the number of 'cheers'
#when snowboy detect the hotword 'how much cheers?(몇 잔이야?)' then, this function is executed.
def howmuch(i):
    if(i==1):
        song = AudioSegment.from_wav("1zzan.wav")
        play(song)
        global interrupted
        #play the wav file and change the interrupted as True
        #when it is True then detector.start() loop is terminated.
        interrupted=True
    elif(i==2):
        song = AudioSegment.from_wav("2zzan.wav")
        play(song)
        interrupted=True
    elif(i==3):
        song = AudioSegment.from_wav("3zzan.wav")
        play(song)
        interrupted=True
    elif(i==4):
        song = AudioSegment.from_wav("4zzan.wav")
        play(song)
        interrupted=True
    elif(i==5):
        song = AudioSegment.from_wav("5zzan.wav")
        play(song)
        interrupted=True
    else:
        song = AudioSegment.from_wav("stopdrinking.wav")
        play(song)
        print('stop drinking!')
        interrupted=True

#when the snowboy detects hotword 'shall we cheers?(우리 짠할까?)', then this function is executed.       
def zzanhalgga(i):
    if(i<6):
        song=AudioSegment.from_wav("goodwork.wav")
        play(song)
        global interrupted
        interrupted=True
    else:
        #if the number of cheers is more than 5, then cheersbot says 'stop drinking!'
        song = AudioSegment.from_wav("stopdrinking.wav")
        play(song)
        print('stop drinking!')
        interrupted=True        
    

In [ ]:
# capture SIGINT signal, e.g., Ctrl+c
signal.signal(signal.SIGINT, signal_handler)

#The bigger the value, the more senstive the decoder. 
#If an empty list is provided, then the default sensitivity in the model will be used.
sensitivity = [0.5]

#detect hotword(pre-trained sentence) using model('howmuch_zzan.pmdl' and 'zzanhalgga.pmdl')
detector = snowboydecoder.HotwordDetector(models, sensitivity=sensitivity)

#each function receives the number of cheers and returns the state of 'interrutped'
callbacks = [lambda: howmuch(i),
             lambda: zzanhalgga(i)]

#initialize the number of cheers(i) as 1
i=1
#There are 10 iterations, you can freely adjust it
while(i!=11):
    if(i<6):
        robot = Robot() #make robot instance
        detections = model(camera.value) #store the results of object detection using the pre-trained pytorch model
        #initialize image number and object number
        image_number = 0
        object_number = 0
        
        #if the detected objects is 1(person's hand) and 47(cup) then it moves forward
        if((detections[image_number][object_number]['label']==1)|(detections[image_number][object_number]['label']==47)):
                                    
            if(i==3):
                song=AudioSegment.from_wav("letszzan.wav")
                play(song)
                
            robot.forward(0.2)
            time.sleep(1.0)
            robot.stop()

            #make the MPU9250 instance to read the MPU9250 sensor
            mpu9250 = FaBo9Axis_MPU9250.MPU9250()
            #read accelorometer data
            accel=mpu9250.readAccel()

            #read the accel data until the value is less than specific thershold
            #we set this threshold through several trials
            #if 'cheers' interaction occurs, then while loop is terminated
            while((abs(accel['x'])<0.2)|(abs(accel['z'])<0.2)):
                accel=mpu9250.readAccel()

            #Print 'Cheers!' to check the accelorometer value is more than threshold
            print('Cheers!')
            #Cheersbot moves backward, and one 'Cheers' interaction is completed
            robot.backward(0.2)
            time.sleep(1.0)
            robot.stop()

            #These are the auditory feedbacs after cheers interaction
            #Depending on the number of cheers, the types of feedback are different
            if(i%2==0):
                song = AudioSegment.from_wav("zzan.wav")
                play(song)
            else:
                song = AudioSegment.from_wav("hardtime.wav")
                play(song)
        
            #The snoyboy detector starts to listen hotword sentences
            #If the sentence is 'How much cheers?(몇 잔?)', then howmuch() function in callbacks is executed, and passed to detected_callback
            #Else if the sentence is 'Shall we cheers?(우리 짠할까?)' then zzanhalgga() function is executed, and passed to detected_callback         
            print('Listening... Press Ctrl+C to exit')
            detector.start(detected_callback=callbacks, interrupt_check=interrupt_callback,sleep_time=0.03)
            #If each function return 'interrupted' as True, then detector.start() loop is terminated
            detector.terminate()
            #Set 'interrupted' as False again for iteration
            interrupted=False
            #count the number of cheers
            i=i+1

        #If the detected object is not person's hand of glass, then the number of cheers is not counted
        #And return to the first part of while loop    
        else: i=i;


    #if the number of cheers is more than 5, then Cheersbot reject my cheers    
    else:
        #When the snoyboy detector detects sentence as 'How much cheers?' or 'Shall we cheers?' then it says 'stop drinking!' 
        print('Listening... Press Ctrl+C to exit')
        detector.start(detected_callback=callbacks, interrupt_check=interrupt_callback,sleep_time=0.03)
        detector.terminate()
        interrupted=False
        #Cheersbot shakes its body
        robot.left(0.3)
        time.sleep(0.5)
        robot.stop()
        robot.right(0.4)
        time.sleep(0.5)
        robot.stop()
        robot.left(0.3)
        time.sleep(0.5)
        robot.stop()
        robot.right(0.4)
        time.sleep(0.5)
        robot.stop()
        i=i+1

1
Cheers!
Listening... Press Ctrl+C to exit


INFO:snowboy:Keyword 2 detected at time: 2020-06-26 03:02:36


1
Cheers!
Listening... Press Ctrl+C to exit


INFO:snowboy:Keyword 2 detected at time: 2020-06-26 03:02:46


1
Cheers!
Listening... Press Ctrl+C to exit


...

Listening... Press Ctrl+C to exit


INFO:snowboy:Keyword 1 detected at time: 2020-06-26 03:03:42


stop drinking!
Listening... Press Ctrl+C to exit


INFO:snowboy:Keyword 2 detected at time: 2020-06-26 03:03:48


stop drinking!


In [ ]:
#If Cheersbot does not stop although above cell is terminated, please execute this cell
#Then it will stop
robot.stop()